#### Importing the required libraries 

In [1]:
from time import sleep
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import csv
import re
import textwrap
from selenium.webdriver.chrome.service import Service

#### Setting chrome options ; instructs chrome to run headless and tells chrome about which agent to use

In [2]:
def create_options():
   # ua = UserAgent()
    ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    options = Options()
    
    options.add_argument(f"user-agent={ua}")
    # options.add_argument("--headless")
    options.add_argument("--disk-cache-dir=/dev/null")
    options.add_argument("--disk-cache-size=0")
    options.add_argument("--no-sandbox")
    options.add_argument("--no-zygote")
    options.add_argument("--disable-setuid-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    preferences = {
        "webrtc.ip_handling_policy": "disable_non_proxied_udp",
        "webrtc.multiple_routes_enabled": False,
        "webrtc.nonproxied_udp_enabled": False,
    }
    
    options.add_experimental_option("prefs", preferences)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)

    return options
    

#### Function to write the scraped data to csv file 

In [3]:
def format_text(text):
    formatted_text = re.sub(r'[^\w\s-]', '', text)
    return formatted_text    
    return text

def write_adds(data):
    with open('indeed_3.csv', 'w', newline='', encoding="utf8") as f:
        csvwriter = csv.writer(f)
        head = ["Job title",'Company', 'Location', 'Job Description', 'Date posted ', 'Salary', 'qualification']

        csvwriter.writerow(head)
        for i in data:
            csvwriter.writerow(i)
    

#### Installing chrome driver with chrome manager and create an instance of chrome driver 

In [4]:
## Provide directory path of chromeDriver use '/' instead of '\' while giving the path

def get_chromedriver():
    service = Service(r'F:/My Laptop/Techeron//10. Resume job scrapper 27 Apr 2023/chromedriver.exe')  #f:/My Laptop/Techeron//10. Resume job scrapper 27 Apr 2023
    driver = webdriver.Chrome(service=service,options=create_options())
    return driver


#### Function to scrape the required data from website and store it in a list 

In [5]:
def get_data():
    print(f'Getting Data from')
    job_profile = input("Enter the job profile you want to search:: ")
    search_pages = int(input("Enter Number of pages you want to search::  "))
    search_criteria = int(input(textwrap.dedent('''\
                                        Last 24 hours: 1
                                        Last 3 days: 3
                                        Last 7 days: 7
                                        Last 14 days: 14
                                        Enter the Date Posted of jobs:: ''')))

    driver = get_chromedriver()
    datalist = []
    linklist = []
    url = f'https://in.indeed.com'
    driver.get(url)

    # Enter job profile in search field
    search = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="text-input-what"]')))
    search.send_keys(job_profile)

    # Click on search button
    search_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(),"Find jobs")]')))
    search_btn.click()

    # Set date filter
    date_filter = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="filter-dateposted"]')))
    date_filter.click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="filter-dateposted-menu"]/li[1]')))
    try:
        date_filter_options = driver.find_elements(By.XPATH, f'//*[@id="filter-dateposted-menu"]/li[{search_criteria}]')
    except:
        print("Filter option Not availbel Using default -1")
        date_filter_options = driver.find_elements(By.XPATH, f'//*[@id="filter-dateposted-menu"]/li[-1]')
        
    for option in date_filter_options:
        if option.text != '':
            option.click()

    # Close popup if present
    try:
        close_button = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'icl-CloseButton')))
        close_button.click()
    except:
        pass

    # Get all links
    for i in range(search_pages):
        linklist += [link.get_attribute('href') for link in WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@class="resultContent"]/div/h2/a')))]
        try:
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@data-testid ="pagination-page-next"]')))
            next_button.click()
        except:
            pass

    # Scrape data from links    
    for i in linklist:
        try:
            driver.get(i)
            sleep(2)
            try:
                title = driver.find_element(By.XPATH, '//*[contains(@class, "jobsearch-JobInfoHeader-title")]/h1').text
            except:
                continue
#             print(f"Title:: {title}")
            try:
                company = driver.find_element(By.XPATH, "//div[@data-company-name]").find_element(By.XPATH,".//a").text
            except:
                continue
#             print(f"Company:: {company}")
            try:

                location = driver.find_element(By.XPATH,"/html/body/div[1]/div[2]/div/div[4]/div/div/div[1]/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div").text
            except:
                location = "India"
#             print(f"Location:: {location}")
            try:
                description = driver.find_element(By.XPATH, '//*[@id="jobDescriptionText"]').text
            except:
                description = ''
            try:
                date = driver.find_element(By.XPATH, '//div[@class="css-q7fux eu4oa1w0"]//ul[@class="css-659xjq eu4oa1w0"]//li[@class="css-5vsc1i eu4oa1w0"]').text
            except:
                date = ''
#             print(f"Date:: {date}")
            try:
                salary_val = driver.find_element(By.XPATH,'//div[@class="css-4m8ia3 eu4oa1w0"]//div[@class="css-1hplm3f eu4oa1w0"]//div[@class="css-tvvxwd ecydgvn1"]').text
                salary = salary_val if re.search(r'\d+', salary_val) else ""
                salary = format_text(salary)
            except:
                salary = ''
#             print(f"Salary:: {salary}")
            try:
                main_div = driver.find_element(By.CSS_SELECTOR, 'div.css-1s1iei0.eu4oa1w0')
                qualification = [li.find_element(By.CSS_SELECTOR, 'p').text for div in main_div.find_elements(By.CSS_SELECTOR, 'div.css-1kvmpfe.eu4oa1w0') for li in div.find_element(By.CSS_SELECTOR, 'ul').find_elements(By.CSS_SELECTOR, 'li')]

            except:
                qualification = ''
            
            datalist.append([title, company, location, description, date, salary, qualification])
            write_adds(datalist)
        except:
            pass


#### Main function 

In [6]:
if __name__ == '__main__':
    get_data()

Getting Data from
Enter the job profile you want to search:: Java Developer
Enter Number of pages you want to search::  1
Last 24 hours: 1
Last 3 days: 3
Last 7 days: 7
Last 14 days: 14
Enter the Date Posted of jobs:: 14
Title:: Senior Java Developer
Company:: Height8 Technologies Pvt. Ltd.
Location:: Ahmedabad, Gujarat
Date:: Employer reviewed job 2 days ago
Salary:: 600000 - 900000 a year
Title:: Full Stack Developer / TechLead – React.js Redux Java JavaScript
Company:: Infinite Enterprise Consulting
Location:: Ahmedabad, Gujarat
Date:: Posted 4 days ago
Salary:: 1200000 - 1500000 a year
Title:: Software Developer (Java, J2EE)
Company:: nahar technologies
Location:: Bengaluru, Karnataka
Date:: Posted 30+ days ago
Salary:: 
Title:: Java Microsevices Developer
Company:: Infosys
Location:: Bengaluru, Karnataka
Date:: Posted Today
Salary:: 
Title:: Sr. Java Developer
Company:: Publicis Sapient
Location:: Bengaluru, Karnataka
Date:: Posted 30+ days ago
Salary:: 1440000 - 2870000 a year
Ti